In [100]:
import pandas as pd
import os
path_to_results = os.environ['HOME']+'/projects/ra_recommendator_conrec/results/'

# All experiments
# experiment = "20190717_1108_5p_100"
# experiment = "20190714_1211_6p"
experiment = "20190714_1213_5p"
# experiment = "20190714_1231_7p"
# experiment = "20190714_1231_8p"
# experiment = "20190714_1232_9p"
# experiment = "20190714_1233_10p"
# experiment = "20190714_1234_3p"
# experiment = "20190714_1235_4p"
# experiment = "20190714_1236_2p"


nb_of_participants = experiment[14:-1]
scenarios = ['A', 'B', 'C', 'D']
data = {}
for s in scenarios:
    data[s] = pd.read_csv(path_to_results+experiment+'/recall_for_scenario_'+s+'.csv', sep='\t', index_col=0)

data['A'].head()


,top 5,top 10,top 15,top 20,top 25,top 30,top 35,top 40,top 45,top 50,...,top 105,top 110,top 115,top 120,top 125,top 130,top 135,top 140,top 145,top 150
q_id,,,,,,,,,,,,,,,,,,,,,
221356,0.2,0.4,0.4,0.6,0.6,0.6,0.6,0.6,0.6,0.8,...,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8
46681,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
9637,0.6,0.6,0.6,0.6,0.8,0.8,0.8,0.8,0.8,0.8,...,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8
47920,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
143841,0.4,0.4,0.4,0.4,0.4,0.6,0.6,0.6,0.6,0.6,...,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8


All questions in which any of the scenarios performed poorly, ie. Recall@60 <= 0.2

In [99]:
#questions_with_bad_performance_in_previous_results = [9135, 9197, 9251, 9273, 9277, 9427, 9606, 9615, 9620, 9653, 9684, 9750, 9796, 10006, 10013, 10020, 10027, 10072, 10105, 10120, 10149, 10238, 10280, 10284, 10313, 10323, 10342]

Utils: `has_bad_performance`

In [101]:
def has_bad_performance(df,q_id,column,threshold):
    # Example: has_bad_performance(row,'top 5',0.4) >> True
    return df.loc[q_id,column] <= threshold

## Harmful Analysis
How bad B, C or D worked in comparison to a good result in baseline.
Example, for question *9166*, the baseline got a Recall of **1.0** at top 30 users.
On the contrary, the performance for the alternatives were, **B:0.6**, **C: 0.6** (both -40%) and **D:0.2** (**80% decay**)

23/07/2019 16:00 - I run it with these paramteres: `column='top 30'` and `threshold=0.4` over the sample 20190714_1213_5p - 495 questions in total (out of 4768 questions in the sample, 10.38%)

In [102]:
# Harmfulness - Good performance in Scenario A worsen by any of scenarios B|C|D
def harmfulness_analysis(column='top 30',threshold=0.4):
    candidates = []
    
    # get questions with good peformance in Baseline
    for q_id, row in data['A'].iterrows():
        if row[column] > threshold:
            candidates.append(q_id)
    
    questions_with_bad_performance = set()
    # select any of the questions whose baseline performance has been worsen in B, C or D
    for scenario in scenarios[1:]: # we are iterating over results of B, C or D.
        for q_id in candidates:
            if has_bad_performance(data[scenario],q_id,column,threshold):
                questions_with_bad_performance.add(q_id)

    return questions_with_bad_performance


 

## Original bad performance table: 
All questions in which any of the scenarios performed poorly, ie. Recall@60 <= 0.2

In [117]:
questions_with_bad_performance = set()
threshold = 0.2
column = 'top 60'

for scenario in scenarios:
    for q_id, row in data[scenario].iterrows():
        if has_bad_performance(data[scenario],q_id,column,threshold):
            questions_with_bad_performance.add(q_id)



### Table of Performance Improvement/Decay

In [118]:
from math import floor 

bad_performance_table = []
for scenario in scenarios:
    bad_performance_table.append(data[scenario][data[scenario].index.isin(questions_with_bad_performance)][column])
df = pd.DataFrame(bad_performance_table)
df = df.transpose()
df.columns = scenarios

def get_arrows(baseline,result):
    up_arrow   = '⬆'
    down_arrow = '⬇'
    
    diff = result*10 - baseline*10
    breaks = 2 # increments are in deltas of 0.2
    nb_of_arrows = int(abs(diff)/breaks)
    
    if nb_of_arrows == 0:
        return '-'
    
    # nb_of_arrows > 0 
    if diff > 0.0:
        return up_arrow*nb_of_arrows
    elif diff < 0.0:
        return down_arrow*nb_of_arrows
    else: 
        return '-'

# Adding new columns with Improvement/Decay 
for index, row in df.iterrows():   
    df.at[index,'Q+'] = get_arrows(row['A'],row['B'])
    df.at[index,'U+'] = get_arrows(row['A'],row['C'])    
    df.at[index,'Q+ U+'] = get_arrows(row['A'],row['D'])

    
# Reordering columns
cols = ['A','B','Q+','C','U+','D','Q+ U+']
df = df[cols]

df.sort_index()


,A,B,Q+,C,U+,D,Q+ U+
q_id,,,,,,,
9056,0.6,0.4,⬇,0.6,-,0.2,⬇⬇
9084,0.4,0.2,⬇,1.0,⬆⬆⬆,0.4,-
9134,0.4,0.4,-,0.8,⬆⬆,0.2,⬇
9135,0.2,0.2,-,1.0,⬆⬆⬆⬆,0.8,⬆⬆⬆
9208,0.4,0.2,⬇,0.6,⬆,0.6,⬆
9273,0.2,0.2,-,0.6,⬆⬆,0.4,⬆
9277,0.2,0.0,⬇,0.4,⬆,0.4,⬆
9427,0.4,0.2,⬇,1.0,⬆⬆⬆,0.6,⬆
9471,0.4,0.0,⬇⬇,0.4,-,0.0,⬇⬇


In [119]:
path_to_reports = path_to_results = os.environ['HOME']+'/projects/ra_recommendator_conrec/reports/'
df.to_csv(path_to_reports+"bad_results/"+experiment+'_top_'+column[4:]+'_threshold_'+str(threshold)+'.csv')